In [ ]:
import numpy as np

import time

import os

from random import randint

import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable

from torch.utils.data import DataLoader

from torchvision.utils import make_grid
import torchvision.transforms as transforms
from torchvision import datasets

from models.model import Autoencoder
from utils.data import autoloader
from utils.utils import save_ckp

import matplotlib.pyplot as plt
%matplotlib inline

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
path = './dataset/'

A_path = path+'jaime'
B_path = path+'willyrex'

bs = 8
num_workers=2

A_loader, B_loader = autoloader(A_path, B_path, bs=bs)

In [ ]:
#imshow method from: https://pytorch.org/tutorials/beginner/transfer_learning_tutorial.html
def imshow(inp, title=None):
    """Imshow for Tensor."""
    inp = inp.numpy().transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)
    plt.imshow(inp)
    if title is not None:
        plt.title(title)
    plt.pause(0.001)

In [ ]:
#inputs, _ = next(iter(A_dataset))

#imshow(inputs, title='Jaime')

In [ ]:
#inputs, _ = next(iter(B_dataset))

#imshow(inputs, title='Willyrex')

In [ ]:
modelA = Autoencoder().to(device)
modelB = Autoencoder().to(device)

In [ ]:
criterion = nn.MSELoss()

optimizerA = optim.Adam(modelA.parameters(), lr=0.001)
optimizerB = optim.Adam(modelB.parameters(), lr=0.001)

In [ ]:
#optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9, nesterov=True)
#exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.1)

In [ ]:
epochs = 15

lossA_plot = []
lossB_plot = []

save_path = "./checkpoints/"
    
iteration = 0
for epoch in range(epochs):
    start = time.time()
    
    for (A_imgs, _), (B_imgs, _) in zip(A_loader, B_loader):
        
        A = Variable(A_imgs).to(device)
        B = Variable(B_imgs).to(device)

        optimizerA.zero_grad()
        optimizerB.zero_grad()

        outputsA = modelA(A)
        outputsB = modelB(B)

        lossA = criterion(outputsA, A)
        lossB = criterion(outputsB, B)

        lossA.backward()
        lossB.backward()
        
        optimizerA.step()
        optimizerB.step()
        
        lossA_plot.append(lossA.item())
        lossB_plot.append(lossB.item())
        
        if iteration % 500 == 0:
            imshow(make_grid(modelA(A_imgs.cuda()).detach().cpu()[0]), title=str(iteration))

        iteration +=1
            
        
    finish = time.time()
        
    print(f"Epoch {epoch+1}/{epochs} ({round((epoch+1)/epochs * 100, 2)}%) | LossA: {lossA.item()}, LossB: {lossB.item()} | The loop took {round(finish-start, 2)} seconds.")
    
    if (finish-start)*(epochs-epoch) > 60:
        print(f"Estimated time left {round((finish-start)*(epochs-epoch)/60, 2)} minutes.")
    else:
        print(f"Estimated time left {round((finish-start)*(epochs-epoch), 2)} seconds.")
    
    checkpointA = {
        'epoch': epoch + 1,
        'state_dict': modelA.state_dict(),
        'optimizer': optimizerA.state_dict()
    }
    
    checkpointB = {
        'epoch': epoch + 1,
        'state_dict': modelB.state_dict(),
        'optimizer': optimizerB.state_dict()
    }
    
    save_ckp(checkpointA, save_path, f"{save_path}epoch-{epoch+1}(A)")
    
    save_ckp(checkpointB, save_path, f"{save_path}epoch-{epoch+1}(B)")

In [ ]:
plt.plot(lossA_plot)

In [ ]:
plt.plot(lossB_plot)

In [ ]:
imshow(make_grid(modelA(A_imgs.cuda()).detach().cpu()[0]), title=str(iteration))